# Глава 2. Сверточные нейронные сети
## Часть 2. Сегментация изображений

### 1. Мотивация

Большинство людей в сообществах глубокого обучения и компьютерного зрения понимают, что такое классификация изображений: мы хотим, чтобы наша модель сообщала нам, какой отдельный объект или сцена присутствует на изображении. Классификация очень грубая и на высоком уровне.

Многие также знакомы с обнаружением объектов (object detection), где мы пытаемся найти и классифицировать несколько объектов в изображении, рисуя ограничивающие рамки вокруг них, а затем классифицируя то, что находится в этом поле. Обнаружение находится на среднем уровне, где у нас есть довольно полезная и подробная информация, но она все еще немного грубовата, поскольку мы только рисуем ограничивающие рамки и не получаем точного представления о форме объекта.

Семантическая сегментация является наиболее информативной из этих трех, где мы хотим классифицировать каждый пиксель на изображении, как вы видите на рисунке выше! За последние несколько лет это было сделано полностью с глубоким изучением.

В этом руководстве вы познакомитесь с базовой структурой и принципами работы моделей семантической сегментации, а также со всеми новейшими и лучшими современными методами.

Если вы хотите самостоятельно опробовать модели, вы можете ознакомиться с моим Semantic Segmentation Suite, дополненным учебным и тестовым кодом TensorFlow для многих моделей этого руководства!

### 1. Базовая структура моделей

* Базовая структура моделей семантической сегментации, которую я собираюсь показать вам, присутствует во всех современных методах! Это позволяет очень легко реализовать разные, так как почти все они имеют одинаковую базовую основу, настройку и поток.

* Модель U-Net прекрасно иллюстрирует эту структуру. Левая сторона модели представляет собой любую сеть извлечения признаков, подготовленную для классификации изображений. Это включает в себя сети, такие как **VGGNet, ResNets, DenseNets, MobileNets и NASNets**! Вы действительно можете использовать там все, что захотите.

* Главное при выборе вашей сети классификации для извлечения признаков - помнить о компромиссах. Использование очень глубокого **ResNet152** даст вам большую точность, но не будет таким же быстрым, как **MobileNet**. Компромиссы, возникающие при применении этих сетей к классификации, также появляются при использовании их для сегментации. Важно помнить, что эти магистральные сети будут главными факторами при проектировании / выборе вашей сети сегментации, и я не могу этого подчеркнуть.

![title](img/seg7.png)

После извлечения карт признаков они затем обрабатываются на **разных масштабах**. Причина этого двоякая. Во-первых, ваша модель, скорее всего, столкнется с объектами разных размеров; обработка карт признаков на разных масштабах даст сети возможность обрабатывать разные характерные размеры на изображениях.

Во-вторых, при выполнении сегментации существует компромисс. Если вам нужна хорошая точность классификации, то вы определенно захотите обработать высокоуровневые признаки позже в сети, поскольку они более различимы для разных классов и содержат больше полезной семантической информации. С другой стороны, если вы обрабатываете только эти глубокие высокоуровневые признаки, вы не получите хорошую локализацию из-за **низкого разрешения**!

Все последние современные методы следовали приведенной выше структуре извлечения признаков с последующей многомасштабной обработкой. Таким образом, многие из них довольно просты в реализации и непрервном обучении. Выбор того, какой из них использовать, будет зависеть от вашей потребности в точности по сравнению со скоростью / наличием памяти, поскольку все пытаются найти новые методы решения этой проблемы, сохраняя при этом эффективность.

В следующем пошаговом руководстве о современном уровне развития я собираюсь сосредоточиться на новейших методах, поскольку они будут наиболее полезны большинству читателей после понимания базовой структуры, представленной выше. Мы пройдем по грубому хронологическому порядку, который также примерно соответствует прогрессу современного уровня.

### 2. Общая архитектура 

Общая архитектура семантической сегментации может рассматриваться в широком смысле как последовательнось энкодера и декодера:

* Энкодер обычно представляет собой предварительно обученную классификационную сеть, такую как VGG / ResNet, за которой следует сеть декодировщика.

* Задача декодера состоит в том, чтобы семантически проецировать распознаваемые признаки (более низкое разрешение), изученные энкодером, на пространство пикселей (c более высокое разрешение), чтобы получить плотную попиксельную классификацию.

В отличие от классификации, где конечный результат очень глубокой сети является единственной важной вещью, семантическая сегментация требует не только различения на уровне пикселей, но также и механизма для проецирования распознавающих признаков, изученных на разных этапах энкодера в пространство пикселей.

Различные подходы используют разные механизмы как часть механизма декодирования. Давайте рассмотрим 3 основных подхода:

### Transposed Convolution
(a.k.a. deconvolutions or fractionally strided convolutions, транспонированные свертки)

Некоторые источники используют название деконволюция, что неуместно, потому что это не деконволюция. Что еще хуже, деконволюции существуют, но они не распространены в области глубокого обучения. Фактическая деконволюция переворачивает процесс свертки. Представьте, что вы вводите изображение в один сверточный слой. Теперь возьмите вывод, бросьте его в черный ящик и снова получите исходное изображение. Этот черный ящик делает деконволюцию. Это математическая обратная сторона того, что делает сверточный слой.

Транспонированная свертка в чем-то похожа, потому что она дает такое же пространственное разрешение, которое мог бы дать гипотетический деконволюционный слой. Однако фактическая математическая операция, выполняемая со значениями, отличается. Транспонированный сверточный слой выполняет регулярную свертку, но восстанавливает свое пространственное преобразование.

![title](img/seg26.gif)

![title](img/seg27.gif)

Транспонированная свертка этого не делает. Единственное, что общего, это то, что она гарантирует, что на выходе также будет изображение 5х5, при этом все еще выполняется обычная операция свертки. Чтобы добиться этого, нам нужно выполнить некоторые необычные дополнения (fancy padding) для ввода.

Как вы можете себе представить, этот шаг не изменит процедуру сверху. По крайней мере, не относительно числовых значений.

Он просто восстанавливает пространственное разрешение, которое было ранее и выполняет свертку.

Это не может быть математическим обратным, но для архитектур Encoder-Decoder это все еще очень полезно. 

Таким образом, мы можем комбинировать масштабирование изображения (**image upscaling**) со сверткой вместо двух отдельных процессов. 

![title](img/seg28.png)

### 2.1 Region-based Semantic Segmentation

Методы на основе выделения областей обычно следуют пайплайну сегментации с использованием распознавания, который сначала извлекает области произвольной формы из изображения (region-based detection) и описывает их, после чего осуществляет классификацию на основе областей. Во время тестирования предсказания на основе областей преобразуются в предсказания по пикселям, как правило, путем маркировки пикселя в соответствии с областью наивысшей оценки, которая его содержит.

![title](img/seg17.jpeg)

Для большей конкретики, R-CNN сначала использует выборочный поиск (selective search) для извлечения большого количества предложений объектов, а затем применяет CNN для каждого из них.

http://www.cs.cornell.edu/courses/cs7670/2014sp/slides/VisionSeminar14.pdf


Наконец, классифицирует каждый регион, используя **линейные модели SVM.**

По сравнению с традиционными CNN, которые в основном предназначены для классификации изображений, R-CNN может решать более сложные задачи, такие как обнаружение объектов и сегментация изображений, и даже становится одной из важных основ для обеих областей.  Кроме того, R-CNN может быть построен поверх любых эталонных структур CNN, таких как **AlexNet, VGG, GoogLeNet и ResNet.**

![title](img/seg23.png)

![title](img/seg29.png)

### Mask R-CNN

Учитывая, что Faster R-CNN так хорошо работает для детектирования объектов, можем ли мы расширить подход для выполнения попиксельной сегментации?

![title](img/seg19.png)

В Mask R-CNN полностью сверточная сеть (FCN) добавляется поверх карт признаков CNN в Faster R-CNN для генерации маски (вывод сегментации). Обратите внимание, что это происходит параллельно с классификация и регрессия bounding boxes в Faster R-CNN. Источник: https://arxiv.org/abs/1703.06870.

Mask R-CNN делает это, добавляя ветвь к Faster R-CNN, которая выводит двоичную маску, которая сообщает, является ли данный пиксель частью объекта. Ветвь (на белом изображении выше), как и прежде, является просто полностью сверточной сетью поверх карты объектов на основе CNN. Вот его входы и выходы:

* Входы: Карта признаков CNN.
* Выходы: Матрица с единицами 1 во всех местах, где пиксель принадлежит объекту, и 0 в других местах (это называется двоичной маской).

Но авторам Mask R-CNN пришлось внести небольшую корректировку, чтобы заставить этот пайплайн работать

![title](img/seg20.png)

Вместо RoIPool изображение пропускается через RoIAlign, так что области карты объектов, выбранные RoIPool, более точно соответствуют областям исходного изображения. Это необходимо, потому что сегментация на уровне пикселей требует более детального выравнивания, чем ограничивающие рамки. Источник: https://arxiv.org/abs/1703.06870.

При запуске без изменений в исходной архитектуре Faster R-CNN авторы Mask R-CNN поняли, что области карты признаков, выбранные RoIPool, слегка смещены относительно областей исходного изображения.

Поскольку сегментация изображения требует специфичности на уровне пикселей, в отличие от ограничивающих рамок, это, естественно, приводит к существенным неточностям.

Авторы смогли решить эту проблему, умно настроив RoIPool для более точного выравнивания, используя метод, известный как RoIAlign.

![title](img/seg22.jpeg)

Представьте, что у нас есть изображение размером 128х128 и карта признаков размером 25х25. Давайте представим, что нам нужны признаки, соответствующие верхнему левому углу 15x15 пикселей в исходном изображении (см. выше). Как мы можем выбрать эти пиксели на карте признаков?

Мы знаем, что каждый пиксель в исходном изображении соответствует ~ 25/128 пикселей в карте объектов. Чтобы выбрать 15 пикселей из исходного изображения, мы просто выбираем 15 * 25/128 ~ = 2,93 пикселей.

В RoIPool мы округлили бы это и выбрали 2 пикселя, что приведет к небольшому смещению. Однако в RoIAlign мы избегаем такого округления. Вместо этого мы используем билинейную интерполяцию, чтобы получить точное представление о том, что будет в пикселе 2.93. На высоком уровне это то, что позволяет нам избежать смещений, вызванных RoIPool.

После того как эти маски сгенерированы, Mask R-CNN объединяет их с классификацией и ограничивающими рамками из Faster R-CNN, чтобы создать такие удивительно точные сегментации:

![title](img/seg21.png)

### 2.2 Fully Convolutional Network-Based Semantic Segmentation

Полностью сверточная сеть (FCN) изучает отображение от пикселей к пикселям без извлечения предложений по регионам. 

Пайплайн FCN является продолжением классического CNN. Основная идея заключается в том, чтобы заставить классический CNN принимать в качестве входных данных изображения произвольного размера. Ограничение CNN - принимать изображения фиксированного размера из-за использования полносвязных слоев, которые являются фиксированными. 

В отличие от них, FCN имеют только сверточные и субдискретизующие слои пулинга, которые дают им возможность делать прогнозы на входах произвольного размера.

![title](img/seg18.png)

Одна из проблем в полностью сверточных сетях (FCN) заключается в том, что при распространении через несколько чередующихся сверточных и субдискретизующих слоев разрешение выходных карт признаков снижается.

Поэтому прямые прогнозы FCN обычно имеют низкое разрешение, что приводит к относительно нечетким границам объектов. 

Для решения этой проблемы было предложено множество более продвинутых подходов на основе FCN, в том числе SegNet, DeepLab-CRF и Dilated Convolutions.

### 2.3  Weakly Supervised Semantic Segmentation

Большинство соответствующих методов в семантической сегментации полагаются на большое количество изображений с пиксельными масками сегментации. Однако ручное комментирование этих масок является довольно трудоемким, разочаровывающим и коммерчески дорогим. Поэтому в последнее время были предложены некоторые слабо контролируемые методы (weakly supervised), предназначенные для выполнения семантической сегментации с использованием аннотированных ограничивающих рамок.

![title](img/seg19.jpeg)

Например, Boxsup использовал аннотации bounding boxes в качестве контроля (обучения с учителем) для обучения сети и итеративного улучшения (уточнения) оценочных масок для семантической сегментации. 

Литература

https://www.cv-foundation.org/openaccess/content_iccv_2015/papers/Dai_BoxSup_Exploiting_Bounding_ICCV_2015_paper.pdf

### 2. SOTA

### 2.1 Fully-resolution Resudial Networks (FRRN)

Остаточные сети с полным разрешением (FRRN)

Модель FRRN является очень наглядным примером многоуровневой технологии обработки. Это достигается с помощью 2 отдельных потоков: **residual stream** и **pooling stream**.

Мы хотим обработать семантические признаки для более  **высокой точности классификации**, поэтому FRRN постепенно обрабатывает и сокращает карты признаков в потоке субдискретизации (пулинга). 

В то же время FRRN обрабатывает карты признаков с полным разрешением в остаточном потоке (residual stream).

Таким образом, субдискретизующий поток обрабатывает семантическую информацию высокого уровня (**для высокой точности классификации**), а остаточный поток обрабатывает информацию пикселей низкого уровня (**для высокой точности локализации**)!

Теперь, когда мы обучаем end-to-end сеть, мы не хотим, чтобы эти два потока были полностью отключены. Таким образом, после каждого этапа субдискретизации FRRN выполняет некоторую совместную обработку карт признаков из двух потоков, чтобы **объединить информацию из них.**

![title](img/seg8.png)

### 2.2 Pyramid Scene Parsing Network (PSPNet)

Fully-resolution Residual Network может успешно выполнять многомасштабную обработку данных.

Однако выполнение тяжелой обработки на любом масштабе требует больших вычислительных ресурсов. Более того, FRRN выполняет некоторую обработку в полном разрешении, что очень медленно!

PSPNet предлагает умный способ обойти это, используя несколько масштабов пулинга. Он начинается со стандартной сети извлечения признаков (ResNet, DenseNet и т. д.) И использует функции третьей понижающей дискретизации для дальнейшей обработки.


Литература 

https://arxiv.org/pdf/1612.01105.pdf

Чтобы получить многомасштабную информацию, PSPNet применяет 4 различные операции Max Pooling с 4 различными размерами окна (window sizes) и шагами (strides).

Это эффективно собирает информацию о признаках из 4 различных масштабов без необходимости интенсивной индивидуальной обработки каждого из них! Мы просто делаем легкую свертку для каждой из них после, увеличиваем, чтобы каждая карта объектов имела одинаковое разрешение, и объединяем их все.

Вуаля! Мы объединили многомасштабные карты признаков, не применяя много сверток на них!

* Все это делается на признаках с более низким разрешением для высокой скорости. 

* В конце мы масштабируем карту выходной сегментации до желаемого размера, используя билинейную интерполяцию. Этот метод апскейлинга только после того, как вся обработка выполнена, присутствует во многих современных работах

![title](img/seg9.png)

### 2.3 Сто слоистых тирамису (FCDenseNet)

Если есть одна удивительная тенденция, вызванная глубоким обучением, это потрясающие названия исследовательских работ! В сотне слоев Tiramisu FCDenseNet (звучит восхитительно!) Используется структура, аналогичная архитектуре U-Net, которую мы видели ранее. Основным вкладом является умное использование плотных связей, аналогичное модели классификации DenseNet.

Это действительно подчеркивает сильную тенденцию в компьютерном зрении, где внешний интерфейс извлечения функций является основной основой для успешной работы в любой другой задаче. Таким образом, первое место, где нужно искать повышение точности, - это часто ваш внешний интерфейс.

![title](img/seg25.png)

### 2.4 Переосмысление расширяющихся сверток Atrous / dilated convolutions (DeepLabV3)

DeepLabV3 и еще один умный способ сделать многомасштабную обработку, на этот раз без увеличения параметров.

Эта модель очень легкая. Мы снова начнем с интерфейса извлечения признаков, взяв функции 4-й понижающей дискретизации для дальнейшей обработки. Это разрешение очень низкое (в 16 раз меньше входного), и поэтому было бы здорово, если бы мы могли просто обработать здесь! Сложность в том, что при таком низком разрешении трудно получить хорошую локализацию из-за низкой точности пикселей.

Вот в чем заключается основной вклад DeepLabV3, благодаря умному использованию извилистых сверток (atrous convolutions). Регулярные свертки могут обрабатывать только локальную информацию, так как веса всегда находятся рядом друг с другом. Например, в стандартной свертке 3х3 расстояние между одним весом и любым другим составляет всего один шаг / пиксель.

![title](img/seg24.gif)

Расширяющиеся свертки вводят в сверточные слои еще один параметр, называемый степенью расширения. Это определяет интервал между значениями в ядре. Ядро 3x3 с коэффициентом расширения 2 будет иметь то же поле зрения, что и ядро 5x5, используя только 9 параметров.

Это обеспечивает более широкое поле зрения при той же вычислительной трудоемкости. Расширенные свертки особенно популярны в сегментации в реальном времени. Используйте их, если вам нужно широкое поле зрения, и вы не можете позволить себе много сверток или более крупных ядер.

При свернутых сверлениях мы будем напрямую увеличивать расстояние между сверточными весами, фактически не увеличивая количество взвешиваний в операции. Таким образом, мы все еще используем 3x3 с 9 общими параметрами, мы просто разносим веса, с которыми мы умножаемся дальше друг от друга! Расстояние между каждым весом называется степенью расширения. Диаграмма модели ниже хорошо иллюстрирует эту идею.

![title](img/seg11.png)

Когда мы используем низкий коэффициент расширения, мы будем обрабатывать очень локальную / низкомасштабную информацию. 

Когда мы используем высокую степень расширения, мы обрабатываем более глобальную / крупномасштабную информацию. Таким образом, модель DeepLabV3 сочетает в себе извилистые извилины с различными степенями расширения для сбора информации нескольких масштабов.

Техника апскейлинга в конце после всей обработки, как было объяснено для PSPNet, также выполняется здесь.

### 2.5 Multi-Path Refinement Networks (RefineNet)

Ранее мы показали как FRRN способна к непосредственному объединению информации из нескольких разрешений и объединению этой информации воедино. 

Недостатком было то, что обработка с таким высоким разрешением требовала значительных вычислительных ресурсов, и нам все еще приходилось обрабатывать и комбинировать эти функции с низкими разрешениями!

Модель RefineNet говорит, что нам не нужно этого делать. Когда мы запускаем наше входное изображение через сеть извлечения карт признаков, мы, естественно, получаем многомасштабные карты после каждой понижающей дискретизации (downsamling).

![title](img/seg12.png)

Затем RefineNet обрабатывает эти карты признаков с несколькими разрешениями по принципу «снизу вверх», чтобы объединить информацию c нескольких масштабов.

Во-первых, каждая карта признаков обрабатывается независимо. Затем, когда мы увеличиваем масштаб, мы объединяем карту объектов с низким разрешением с картой с более высоким разрешением, выполняя некоторую дальнейшую обработку для них обоих вместе. 

Таким образом, многомасштабные карты объектов обрабатываются как независимо, так и вместе. Весь процесс движется слева направо на диаграмме ниже.

Техника апскейлинга в конце после всей обработки, как было объяснено для PSPNet и DeepLabV3, также выполняется здесь.

### 2.6 Large Kernel Matters (GCN)
Ранее мы видели, как модель DeepLabV3 использовала atrous / dilated convolutions (расширенные свертки) с различной степенью расширения для сбора многомасштабной информации.

Сложность в том, что мы можем обрабатывать только один масштаб за раз, а затем комбинировать их позже. Например, atrous convolution с инкрементов 16, например, не очень хорошо справляется с локальной информацией и должна быть позже объединена с информацией из свертки с гораздо меньшим инкрементом для применения в семантической сегментации

Таким образом, в предыдущих вариантах, многомасштабная обработка сначала выполняется отдельно, а затем результаты объединяются.

Было бы больше смысла, если бы мы могли получить многомасштабную информацию за один раз.

Чтобы сделать это, Глобальная Сверточная Сеть (GCN) умно предлагает использовать большие одномерные ядра вместо квадратных. С квадратными свертками, такими как 3x3, 7x7 и т.д., мы не сможем сделать их слишком большими без огромного снижения скорости и потребления памяти. С другой стороны, одномерные ядра масштабируются гораздо эффективнее, и мы можем сделать их достаточно большими, не сильно замедляя работу сети. В статье дошли до размера 15!

Важная вещь, которую вы должны обязательно сделать, это сбалансировать горизонтальную и вертикальную свертки. Кроме того, в статье используются маленькие свертки 3х3 с небольшим количеством фильтров для эффективного уточнения всего, что могут пропустить одномерные свертки.

GCN следует тому же стилю, что и предыдущие работы, обрабатывая каждый масштаб из интерфейса извлечения признаков.

Из-за эффективности одномерных сверток GCN выполняет обработку на всех масштабах вплоть до полного разрешения, а не остается маленьким и масштабируется (upscaling) впоследствии. 

Это учитывает постоянное уточнение сегментации по мере увеличения масштаба, вместо возможного bottlenecking, которое может произойти, если мы сохраним более низкое разрешение.

![title](img/seg13.png)

### 2.7 DeepLabV3 +

Модель **DeepLabV3+**, как следует из названия, является улучшением DeepLabV3, заимствуя некоторые концептуальные идеи из достижений, которые были до нее. 

Как мы видели ранее, существует потенциальное узкое место, если мы просто подождем повышения с билинейной интерполяцией в конце сети. Фактически, результат оригинальной модели DeepLabV3 был увеличен до x16 на выходе!

Для решения этой проблемы DeepLabV3 + предлагает добавить промежуточный модуль декодера поверх DeepLabV3. 

После обработки через DeepLabV3 признаки затем подвергаются дискретизации (upsampling) с увеличением x4. Затем они дополнительно обрабатываются вместе с исходными признаками из интерфейса извлечения признаков, а затем снова увеличиваются в 4 раза.\

Это снижает нагрузку на конец сети и обеспечивает быстрый путь (shortcut connection) от внешнего интерфейса извлечения карт признаков до ближайшего конца сети.

![title](img/seg14.png)

### 2.8 CVPR и ECCV 2018

Сети, которые мы рассмотрели в предыдущем разделе, представляют большую часть техник, которые вам необходимо знать для выполнения семантической сегментации!

Многое из того, что было выпущено в этом году на конференциях по компьютерному зрению, было незначительными обновлениями и небольшими изменениями в точности, что не очень важно для начала работы. 

Для тщательности я предоставляю краткий обзор их вкладов здесь для всех, кто заинтересован!

**1. Image Cascade Network (ICNet)** - управляет входным изображением на разных масштабах, каждый из которых масштабируется через собственную подсеть и постепенно объединяет результаты

https://arxiv.org/pdf/1704.08545.pdf

**2. Discriminative Feature Network (DFN)** - использует глубокий контроль и пытается обрабатывать гладкие и краевые части сегментов отдельно

http://openaccess.thecvf.com/content_cvpr_2018/papers/Yu_Learning_a_Discriminative_CVPR_2018_paper.pdf

**3. DenseASPP** - сочетает в себе плотные соединения (dense connections) с atrous convolutions.

http://openaccess.thecvf.com/content_cvpr_2018/papers/Yang_DenseASPP_for_Semantic_CVPR_2018_paper.pdf

**4. Context encoding** - использует глобальный контекст (global context) для повышения точности, добавляя модуль внимания канала (channel attention module), который вызывает внимание на определенных картах признаков, основанных на недавно разработанной функции потерь. Функция потерь основана на ветви сети, которая предсказывает, какие классы присутствуют в изображении (то есть глобальный контекст более высокого уровня).

http://openaccess.thecvf.com/content_cvpr_2018/papers/Zhang_Context_Encoding_for_CVPR_2018_paper.pdf


**5. Dense Decoder Shortcut Connections** - используются плотные связи (dense connections) на этапе декодирования для более высокой точности (ранее это делалось только во время извлечения / кодирования признаков)

http://openaccess.thecvf.com/content_cvpr_2018/papers/Bilinski_Dense_Decoder_Shortcut_CVPR_2018_paper.pdf

**6. Bilateral Segmentation Network (BiSeNet)** - имеет 2 ветви: одна глубокая - для получения семантической информации, а другая - выполняет очень незначительную / минорную обработку входного изображения, чтобы сохранить низкоуровневую информацию о пикселях

https://arxiv.org/pdf/1808.00897v1.pdf

**7. ExFuse** - явно комбинирует многомасштабные признаки от внешнего интерфейса трансферлернинга для извелчения признаков до обработки, чтобы гарантировать, что многомасштабная информация обрабатывается вместе на всех уровнях.

http://openaccess.thecvf.com/content_ECCV_2018/papers/Zhenli_Zhang_ExFuse_Enhancing_Feature_ECCV_2018_paper.pdf

![title](img/seg16.png)

### 3. Key points

* Будьте в курсе преимуществ и недостатков модели классификации. Ваша классификационная сеть является вашим основным драйвером для получения признаков из изобразения, и большая часть ваших выигрышей / потерь будет происходить отсюда

* Обрабатывайте карты признаков на разных масштабах и объединяйте информацию

* Многомасштабный пулинг, atrous convolutions, большие одномерные свертки хороши для семантической сегментации

* Вам не нужно много обрабатывать в высоком разрешении. Делайте большую часть при низком разрешении для скорости, затем увеличивайте масштаб (upscaling) и выполняйте более легкую обработку в конце, в случае необходимости

* Deep supervision может немного повысить вашу точность (хотя и сложнее настроить обучение)

### Ссылки 

https://github.com/GeorgeSeif/Semantic-Segmentation-Suite

https://medium.com/@keremturgutlu/semantic-segmentation-u-net-part-1-d8d6f6005066